In [19]:
import os
import sys
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from utils.beh_functions import parseSessionID, session_dirs, plot_session_in_time_all, plot_session_glm, transfer_nwb, get_unit_tbl
from utils.ephys_functions import make_summary_unit_tbl
from utils.opto_utils import opto_metrics, load_opto_sig
from unit_beh_analysis import plot_unit_beh_session, plot_alignments, burst_analysis
from behavior_and_time_alignment import beh_and_time_alignment_hopkins
import shutil
from aind_dynamic_foraging_data_utils.nwb_utils import load_nwb_from_filename
import pandas as pd
from joblib import Parallel, delayed
from aind_dynamic_foraging_basic_analysis.plot.plot_foraging_session import plot_foraging_session, plot_foraging_session_nwb
from aind_dynamic_foraging_basic_analysis.licks.lick_analysis import plot_lick_analysis
import datetime
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle

In [ ]:
session_id = 'behavior_ZS059_2021-04-12_14-57-43'
data_type = 'curated'
session_id_example   = 'behavior_754897_2025-03-13_11-20-42'
session_dir = session_dirs(session_id)
session_dir['ephys_mat']

Old data, using hopkins formats


'/root/capsule/data/behavior_ZS061_2021-04-17_16-50-25_raw_data/ecephys/sorted/session/mZS061d20210417_sessionData_nL.mat'

In [3]:
opto_metrics_session = opto_metrics(session_id_example, 'curated')
opto_sigs_session = load_opto_sig(session_id_example, 'curated')

Selected experiment1 recording1, length:5856.14
Selected experiment1 recording1, length:5856.14


In [4]:
# make fake opto_metrcs for hopkins data


In [12]:
session_id = 'behavior_ZS061_2021-04-17_16-50-25'
data_type = 'curated'
def convert_hopkins_opto(session_id, data_type='curated'):
    session_dir = session_dirs(session_id)
    unit_nwb = load_nwb_from_filename(session_dir['nwb_dir_curated'])
    unit_tbl = unit_nwb.units[:]
    # opto_metrics
    cols = ['unit_id', 'p_resp', 'lat_resp', 'euc_max_p', 'corr_max_p', 'pulse_width', 'bl_max_p']
    all_neurons = []
    # ['bl_max_p'] 
    for unit_ind, row in unit_tbl.iterrows():
        curr_dict = row[cols].to_dict()
        curr_dict = {
            key: (value[0] if isinstance(value, np.ndarray) else value)
            for key, value in curr_dict.items()
        }
        curr_df = pd.DataFrame(curr_dict)
        curr_df['resp_p_bl'] = curr_df['p_resp'] - curr_df['bl_max_p']
        curr_df.drop(['bl_max_p'], axis=1, inplace=True)
        all_neurons.append(curr_df)
    opto_metrics_tbl = pd.concat(all_neurons, axis=0, ignore_index=True)
    opto_file = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session_id}_opto_tagging_metrics.pkl')
    # with open(unit_tbl_dir, 'rb') as f:
    #     unit_data = pickle.load(f)
    # qm_data = unit_data['opto_tagging_df_metrics']
    opto_metrics_tbl.rename(columns={'p_resp': 'resp_p', 'lat_resp': 'resp_lat', 'euc_max_p': 'euclidean_norm', 'corr_max_p': 'correlation'}, inplace=True)
    opto_tagging_data = {'opto_tagging_df': None, 'opto_tagging_df_metrics': opto_metrics_tbl}
    with open(opto_file, 'wb') as f:
        pickle.dump(opto_tagging_data, f)
    # opto sigs
    opto_sigs_file = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session_id}_opto_sigs.pkl')
    neuron_sig_df = pd.DataFrame({'unit_id': unit_tbl['unit_id'].to_list(), 'p_sig_count': [5]*len(unit_tbl)})
    with open(opto_sigs_file, 'wb') as f:
        pickle.dump(neuron_sig_df, f)

In [15]:
opto_sigs_session = load_opto_sig(session_id, data_type)
opto_sigs_session.opto_sigs

Old data, using hopkins formats


,unit_id,p_sig_count
0,TT3_SS_02,5
1,TT4_SS_02,5


In [17]:
# transfer behavior nwb
opto_metrics_tbl = opto_metrics(session_id, data_type).qm_data
opto_metrics_tbl

Old data, using hopkins formats


,unit_id,resp_p,resp_lat,euclidean_norm,correlation,pulse_width,resp_p_bl
0,TT3_SS_02,0.1,0.000825,NaN,0.677460,10.0,0.100000
1,TT3_SS_02,0.9,0.001048,NaN,0.677460,10.0,0.900000
2,TT3_SS_02,1.0,0.001166,NaN,0.677460,10.0,1.000000
3,TT3_SS_02,1.0,0.001232,NaN,0.677460,10.0,1.000000
4,TT3_SS_02,0.9,0.001252,NaN,0.677460,10.0,0.900000
5,TT3_SS_02,0.8,0.001259,NaN,0.677460,10.0,0.800000
6,TT3_SS_02,0.9,0.001291,NaN,0.677460,10.0,0.900000
7,TT3_SS_02,0.8,0.001306,NaN,0.677460,10.0,0.800000
8,TT3_SS_02,0.6,0.001305,NaN,0.677460,10.0,0.600000
9,TT3_SS_02,0.9,0.001329,NaN,0.677460,10.0,0.900000


In [4]:
# check time alignment
beh_and_time_alignment_hopkins(session_id)

Old data, using hopkins formats
behavior_ZS061_2021-04-17_16-50-25


In [12]:
# make summary unit table
make_summary_unit_tbl(session_id)

Old data, using hopkins formats


In [ ]:
# get summary unit table's columns
example_tbl = get_unit_tbl('behavior_761038_2025-04-15_10-25-11', 'curated')
col_list = example_tbl.columns.tolist()
with open("summary_col_list.json", "w") as f:
    json.dump(col_list, f)

Selected experiment1 recording1, length:3362.95


In [2]:
plot_unit_beh_session('behavior_ZS061_2021-04-20_15-49-57', align_name='go_cue')

Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Plotting unit TT4_SS_01


/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Old data, using hopkins formats
Combining behavior_ZS061_2021-04-20_15-49-57
Processing 2 files in /root/capsule/scratch/ZS061/behavior_ZS061_2021-04-20_15-49-57/ephys/curated/figures/go_cue
Combined PDF saved as: /root/capsule/scratch/ZS061/behavior_ZS061_2021-04-20_15-49-57/ephys/curated/behavior_ZS061_2021-04-20_15-49-57_unit_beh_go_cue.pdf


In [4]:
# check time alignment
beh_and_time_alignment_hopkins(session_id)

Old data, using hopkins formats
behavior_ZS061_2021-04-17_16-50-25


In [5]:
# make summary unit table
make_summary_unit_tbl(session_id)

Old data, using hopkins formats
Selected experiment1 recording1, length:3362.95
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Plotting unit TT4_SS_01
Old data, using hopkins formats
Combining behavior_ZS061_2021-04-20_15-49-57
Processing 2 files in /root/capsule/scratch/ZS061/behavior_ZS061_2021-04-20_15-49-57/ephys/curated/figures/go_cue
Combined PDF saved as: /root/capsule/scratch/ZS061/behavior_ZS061_2021-04-20_15-49-57/ephys/curated/behavior_ZS061_2021-04-20_15-49-57_unit_beh_go_cue.pdf
Old data, using hopkins formats
behavior_ZS061_2021-03-18_17-39-19
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS059_2021-03-29_14-41-57
Processed behavior_ZS059_2021-03-29_14-41-57
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS059_2021

In [6]:
# get summary unit table's columns
example_tbl = get_unit_tbl('behavior_761038_2025-04-15_10-25-11', 'curated')
col_list = example_tbl.columns.tolist()
with open("summary_col_list.json", "w") as f:
    json.dump(col_list, f)

In [7]:
plot_unit_beh_session('behavior_ZS061_2021-04-20_15-49-57', align_name='go_cue')

/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [20]:
# get session list
session_df = pd.read_csv('/root/capsule/code/data_management/hopkins_session_assets.csv')
session_ids = session_df['session_id'].to_list()

In [22]:
def process(session_id):
    session_dir = session_dirs(session_id);
    if os.path.exists(session_dir['nwb_dir_raw']) and get_unit_tbl(session_id, 'curated') is not None:
        # try:
        # transfer_nwb(session_id);
        # beh_and_time_alignment_hopkins(session_id, ephys_cut=[0, 0]);
        # make_summary_unit_tbl(session)
        # plot_unit_beh_session(session_id, align_name='go_cue')
        # plot_unit_beh_session(session_id, align_name='response')
        # plot_alignments(session_id)
        burst_analysis(session_id, 'curated')
        print(f'Processed {session_id}')
        plt.close('all')
        # except:
        #     print(f'{session_id} failed')

In [30]:
for session_ind, session_id  in enumerate(session_ids):
    session_dir = session_dirs(session_id);
    if session_ind >= 14:
        if os.path.exists(session_dir['nwb_dir_raw']):
            process(session_id)

Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Processing session behavior_ZS059_2021-04-10_15-55-11 for data type curated
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS059_2021-04-10_15-55-11 Combining PDFs...
Processing 2 files in /root/capsule/scratch/ZS059/behavior_ZS059_2021-04-10_15-55-11/ephys/curated/figures/burst
Combined PDF saved as: /root/capsule/scratch/ZS059/behavior_ZS059_2021-04-10_15-55-11/

/root/capsule/code/beh_ephys_analysis/unit_beh_analysis.py:570: RuntimeWarning: divide by zero encountered in log
  isi_spikes = np.log(np.diff(spike_times_curr))
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:169: RuntimeWarning: invalid value encountered in multiply
  y *= step
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:179: RuntimeWarning: invalid value encountered in add
  y += start


behavior_ZS059_2021-04-21_15-58-55 Combining PDFs...
Processing 2 files in /root/capsule/scratch/ZS059/behavior_ZS059_2021-04-21_15-58-55/ephys/curated/figures/burst
Combined PDF saved as: /root/capsule/scratch/ZS059/behavior_ZS059_2021-04-21_15-58-55/ephys/curated/figures/behavior_ZS059_2021-04-21_15-58-55_bursting.pdf
behavior_ZS059_2021-04-21_15-58-55 Done!
Processed behavior_ZS059_2021-04-21_15-58-55
Old data, using hopkins formats
No raw session directory found for behavior_ZS059_2021-04-18_17-26-19.
Old data, using hopkins formats
No raw session directory found for behavior_ZS059_2021-04-18_17-26-19.
Old data, using hopkins formats
No raw session directory found for behavior_ZS059_2021-04-18_17-26-19.
No unit table found for behavior_ZS059_2021-04-18_17-26-19 in curated data.
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
No unit table found for behavior_ZS059_2021-04-19_14-42-16 in curated data.
Old data, using hopkins formats
Old

/root/capsule/code/beh_ephys_analysis/unit_beh_analysis.py:570: RuntimeWarning: divide by zero encountered in log
  isi_spikes = np.log(np.diff(spike_times_curr))
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:169: RuntimeWarning: invalid value encountered in multiply
  y *= step
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:179: RuntimeWarning: invalid value encountered in add
  y += start


Old data, using hopkins formats
Old data, using hopkins formats


...

behavior_ZS062_2021-04-13_19-35-13 Combining PDFs...
Processing 2 files in /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-13_19-35-13/ephys/curated/figures/burst
Combined PDF saved as: /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-13_19-35-13/ephys/curated/figures/behavior_ZS062_2021-04-13_19-35-13_bursting.pdf
behavior_ZS062_2021-04-13_19-35-13 Done!
Processed behavior_ZS062_2021-04-13_19-35-13
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Processing session behavior_ZS062_2021-04-14_17-07-51 for data type curated
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS062_2021-04-14_17-07-51 Combining PDFs...
Processing 2 files in /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-14_17-07-51/ephys/curated/figures/burst
Combined PDF saved as: /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-14_17-07-51/ephys/curated/figures/behavior_ZS062_202

/root/capsule/code/beh_ephys_analysis/unit_beh_analysis.py:570: RuntimeWarning: divide by zero encountered in log
  isi_spikes = np.log(np.diff(spike_times_curr))
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:169: RuntimeWarning: invalid value encountered in multiply
  y *= step
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:179: RuntimeWarning: invalid value encountered in add
  y += start


behavior_ZS062_2021-04-17_18-06-40 Combining PDFs...
Processing 3 files in /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-17_18-06-40/ephys/curated/figures/burst
Combined PDF saved as: /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-17_18-06-40/ephys/curated/figures/behavior_ZS062_2021-04-17_18-06-40_bursting.pdf
behavior_ZS062_2021-04-17_18-06-40 Done!
Processed behavior_ZS062_2021-04-17_18-06-40
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Processing session behavior_ZS062_2021-04-18_19-52-39 for data type curated
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS062_2021-04-18_19-52-39 Combining PDFs...
Processing 3 files in /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-18_19-52-39/ephys/curated/figures/burst
Combined PDF saved as: /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-18_19-52-39/ephys/cu

/root/capsule/code/beh_ephys_analysis/unit_beh_analysis.py:570: RuntimeWarning: divide by zero encountered in log
  isi_spikes = np.log(np.diff(spike_times_curr))
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:169: RuntimeWarning: invalid value encountered in multiply
  y *= step
/opt/conda/lib/python3.10/site-packages/numpy/_core/function_base.py:179: RuntimeWarning: invalid value encountered in add
  y += start


Old data, using hopkins formats
No raw session directory found for behavior_ZS062_2021-04-10_19-16-30.
behavior_ZS062_2021-04-10_19-16-30 Combining PDFs...
Processing 3 files in /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-10_19-16-30/ephys/curated/figures/burst
Combined PDF saved as: /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-10_19-16-30/ephys/curated/figures/behavior_ZS062_2021-04-10_19-16-30_bursting.pdf
behavior_ZS062_2021-04-10_19-16-30 Done!
Processed behavior_ZS062_2021-04-10_19-16-30
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Processing session behavior_ZS062_2021-04-19_19-36-48 for data type curated
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS062_2021-04-19_19-36-48 Combining PDFs...
Processing 3 files in /root/capsule/scratch/ZS062/behavior_ZS062_2021-04-19_19-36-48/ephys/curated/figur

In [4]:
# Parallel(n_jobs=5)(delayed(process)(session, data_type) for session in session_list[82:94])
Parallel(n_jobs=-1)(delayed(process)(session_id) for session_id in session_ids)

Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
No unit table found for behavior_ZS059_2021-04-02_15-05-16 in curated data.
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
No unit table found for behavior_ZS059_2021-04-05_14-33-48 in curated data.
Old data, using hopkins formats
Old data, using 

/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Old data, using hopkins formats
Plotting unit TT3_SS_01
Old data, using hopkins formats
Old data, using hopkins formats
Plotting unit TT4_SS_02


/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/opt/conda/lib/p

Old data, using hopkins formats
Combining behavior_ZS059_2021-04-12_14-57-43
Processing 2 files in /root/capsule/scratch/ZS059/behavior_ZS059_2021-04-12_14-57-43/ephys/curated/figures/go_cue


...

/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Plotting unit TT4_SS_04
Plotting unit TT4_SS_01
Old data, using hopkins formats
Old data, using hopkins formatsOld data, using hopkins formats
Old data, using hopkins formats

Old data, using hopkins formats
Plotting unit TT3_SS_02
Plotting unit TT4_SS_01
Plotting unit TT3_SS_03
Plotting unit TT3_SS_01
Old data, using hopkins formats
Combined PDF saved as: /root/capsule/scratch/ZS059/behavior_ZS059_2021-04-23_14-57-50/ephys/curated/behavior_ZS059_2021-04-23_14-57-50_unit_beh_response.pdf
Processed behavior_ZS059_2021-04-23_14-57-50
Old data, using hopkins formats
Old data, using hopkins formatsOld data, using hopkins formats

Old data, using hopkins formatsOld data, using hopkins formats

Old data, using hopkins formatsOld data, using hopkins formats
Old data, using hopkins formats

Old data, using hopkins formats


/opt/conda/lib/python3.10/site-packages/aind_dynamic_foraging_basic_analysis/plot/plot_foraging_session.py:497: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats


KeyError: 'pe'